In [93]:
import sys
!{sys.executable} -m pip install -U folium

Defaulting to user installation because normal site-packages is not writeable


In [94]:
import folium
import pandas as pd

In [95]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [96]:
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(URL)

spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [97]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [98]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [99]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(
    nasa_coordinate, radius=1000, color='#d35400', fill=True
).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    ),
)

site_map.add_child(circle)
site_map.add_child(marker)

In [100]:
# Create and add folium.Circle and folium.Marker for each launch site on the site map

for _, row in launch_sites_df.iterrows():
    coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']

    folium.Circle(
        coordinate,
        radius=1000,
        color='#000000',
        fill=True,
        fill_color='#3186cc'
    ).add_child(folium.Popup(site_name)).add_to(site_map)

    folium.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{site_name}</b></div>',
        )
    ).add_to(site_map)

site_map

In [84]:
# Initial the map

site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [85]:
# create a MarkerCluster object

marker_cluster = MarkerCluster()

In [86]:
# Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value

spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
spacex_df.head()

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red


In [87]:
# For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster

from folium.plugins import MarkerCluster
from folium import Icon, Marker

site_map.add_child(marker_cluster)
MousePosition().add_to(site_map)

for _, row in spacex_df.iterrows():
    coord = [row['Lat'], row['Long']]

    m = folium.Marker(
        location=coord,
        tooltip=row.get('Launch Site', ''),
        popup=f"{row.get('Launch Site','')} — {'Success' if row['class']==1 else 'Failure'}",
        icon=folium.Icon(color='white', icon_color=row['marker_color'], icon='rocket', prefix='fa')
    )

    folium.Tooltip(row.get('Launch Site', ''), permanent=True, direction='right').add_to(m)
    m.add_to(marker_cluster)

site_map

In [88]:
# # Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map

formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [89]:
# Calculate distance function

from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [90]:
# Draw a PolyLine between a launch site to the selected coastline point

try:
    launch_site_lat
    launch_site_lon
except NameError:
    launch_site_lat = float(launch_sites_df['Lat'].iloc[0])
    launch_site_lon = float(launch_sites_df['Long'].iloc[0])

coastline_lat = 28.56211
coastline_lon = -80.57055

distance_km = calculate_distance(
    launch_site_lat, launch_site_lon,
    coastline_lat, coastline_lon
)

folium.Marker(
    [coastline_lat, coastline_lon],
    popup="Closest Coastline",
    icon=folium.Icon(color='blue', icon='flag')
).add_to(site_map)

# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate

coords = [[launch_site_lat, launch_site_lon],
          [coastline_lat, coastline_lon]]
lines = folium.PolyLine(locations=coords, weight=2)
site_map.add_child(lines)

In [92]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

mid_lat = (launch_site_lat + coastline_lat) / 2
mid_lon = (launch_site_lon + coastline_lon) / 2
distance_marker = folium.Marker(
    [mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:,.2f} KM</b></div>'.format(distance_km),
    )
)
distance_marker.add_to(site_map)

site_map

In [91]:
site_map.save("spacex_launch_map.html")